# Billboard Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
from glob import glob
from os import getcwd
from os.path import join
from fileUtils import getBasename, getDirname, getBaseFilename
from fsUtils import isFile, isDir, moveFile, removeFile
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib

from discogs import discogs
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-05-06 21:55:57.845782


In [2]:
disc = discogs()
discdata = disc.getArtistNameToIDsData()

Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Loading data from /Volumes/Music/Discog/db/NameToIDs.p
  --> This file is 24.1MB.
Loading /Volumes/Music/Discog/db/NameToIDs.p


{'Genry': ['3992176'],
 'Ð¡ÐµÑ\x81Ñ\x82Ñ\x80Ñ\x8b Ð\x91ÐµÑ\x80Ð·ÐµÐ½Ð¸Ñ\x8f': ['4956311'],
 'Mr. Erik': ['3992147'],
 'Wilhelm Friedemann Bach': ['891601'],
 'Eric Mikaia': ['4956322'],
 'Ð\x95Ñ\x82Ð½Ð¾Ð³Ñ\x80Ð°Ñ\x84Ð¸Ñ\x87ÐµÑ\x81ÐºÐ¸Ð¹ Ð\x9cÑ\x83Ð¶Ñ\x81ÐºÐ¾Ð¹ Ð¥Ð¾Ñ\x80 "Ð\x90Ð±Ñ\x85Ð°Ð·Ð¸Ñ\x8f"': ['4788717'],
 'La Murga Xicohtl': ['5484857'],
 'Jamiroquai': ['8029'],
 'Paul Winter Consort': ['989059'],
 'Native Americans In Ð¢he United States': ['1808480'],
 'Chill On The Sun': ['2136756'],
 'Nashville Session Players': ['4276041'],
 'Ash Dargan': ['1120037'],
 'Gyasi Ross': ['1940322'],
 'The Marshall Tucker Band': ['507564'],
 'Gawurra': ['5414653'],
 'Kashtin': ['514811'],
 'The White Cockatoo Performing Group': ['5385685'],
 'The Legendary Strawberry Man': ['5024697'],
 'Mark Hoffmann': ['4192308', '3010698'],
 'Pandera': ['157393'],
 'Jones (Band)': ['5369473'],
 'A Tribe Called Red': ['2964848'],
 'De Kast': ['283010'],
 'Dr. Didg': ['98064'],
 'Archie Roach': ['474364'],
 'Medi

In [ ]:
nameids = {}
namerefs = {}
for name,nameid in iddata.items():
    artist = name
    if name.endswith(")"):
        artist = None
        for x in [-3,-4,-5]:
            if artist is not None:
                continue
            if abs(x) > len(name):
                continue
            if name[x] == "(":
                try:
                    val = int(name[(x+1):-1])
                    artist = name[:x].strip()
                except:
                    continue
          
        if artist is None:
            artist = name

    ref = refdata[name]
            
    if nameids.get(artist) is None:
        nameids[artist] = {}
    nameids[artist][nameid] = 1
    
    if namerefs.get(artist) is None:
        namerefs[artist] = {}
    namerefs[artist][ref] = 1
    

nameids = {k: list(v.keys()) for k,v in nameids.items()}
namerefs = {k: list(v.keys()) for k,v in namerefs.items()}

# Starter Class Section

In [4]:
basedir = getcwd()
try:
    filename = glob(join(basedir, "data", "billboard", "starter.html"))[0]
except:
    print("Could not find starter HTML file!")
fdata = getHTML(filename)

In [8]:
chartData  = {}
dirname = None
baseURL = "https://www.billboard.com"
for iul,ul in enumerate(fdata.findAll("ul")):
    lis = ul.findAll("li", {"class": "chart-group__item"})
    for j,li in enumerate(lis):  
        a = li.find('a')
        if a is not None:
            href = a['href']
            text = a.text
            subdir = getDirname(href)[1:]
            chartData[text] = [baseURL, subdir, getBasename(href)]

# Download Yearly Information

In [9]:
from time import sleep
years = [str(x) for x in range(1958,2014)]
for year in years:
    url="https://www.billboard.com/archive/charts/{0}".format(year)
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    
    savedir = join(basedir, "data", "billboard", "yearly")
    savename = join(savedir, "{0}.p".format(year))
    if isFile(savename):
        continue
    
    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    print("Saving {0}".format(savename))
    saveJoblib(data=data, filename=savename, compress=True)
    sleep(1)

# Parse Yearly Information

In [10]:
downloads = {}
baseURL = "https://www.billboard.com"
for ifile in sorted(glob(join(basedir, "data", "billboard", "yearly", "*.p"))):
    year = getBaseFilename(ifile)
    if downloads.get(year) is None:
        downloads[year] = {}
    fdata = getHTML(ifile)
    for iul,ul in enumerate(fdata.findAll("ul")):
        lis = ul.findAll("li", {"class": "chart-group__item"})
        for j,li in enumerate(lis):  
            a = li.find('a')
            if a is not None:
                href = a['href']
                text = a.text
                subdir = getDirname(href)[1:]
                downloads[year][href] = [baseURL, subdir, getBasename(href)]

# Download Category Data

In [11]:
baseURL = "https://www.billboard.com"

for year, yearData in downloads.items():   
    for href, hrefData in yearData.items():
        url  = "{0}/{1}".format(baseURL, href)
        year = getBasename(hrefData[1])
        category = hrefData[2]

        savedir  = join(basedir, "data", "billboard", "categories")
        savename = join(savedir, "{0}-{1}.p".format(year, category))

        if isFile(savename):
            continue

        print("  Trying to download and save {0}".format(savename))
        try:

            user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
            headers={'User-Agent':user_agent,} 

            request=urllib.request.Request(url,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read() # The data u need
        except:
            continue

        print("Saving {0}".format(savename))
        saveJoblib(data=data, filename=savename, compress=True)
        sleep(1)        

  Trying to download and save /Users/tgadfort/Documents/code/charts/data/billboard/categories/2018-billboard-argentina-hot-100.p


# Parse Category Data

In [12]:
data = {}

baseURL = "https://www.billboard.com"
names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
categories = set([x[5:] for x in names])

names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
years = sorted(set([x[:4] for x in names]))

print("There are {0} years".format(len(years)))
print("There are {0} charts".format(len(categories)))

There are 61 years
There are 90 charts


In [14]:
data = {}
for category in sorted(categories):
    categoryData = {}
    for year in sorted(years):
        for ifile in sorted(glob(join(basedir, "data", "billboard", "categories", "{0}-{1}.p".format(year, category)))):
            try:
                fdata = getHTML(ifile)
            except:
                removeFile(ifile, debug=True)
                continue
            for i,table in enumerate(fdata.findAll('table')):
                ths = table.findAll("th")
                trs = table.findAll("tr")

                headers = [x.text for x in ths]
                for j,tr in enumerate(trs[1:]):
                    tds  = tr.findAll('td')
                    vals = [x.text for x in tds]

                    if len(vals) == 3:
                        date,name,artist = vals
                        try:
                            date = getDateTime(", ".join([date, year]))
                        except:
                            print(vals)
                            print(", ".join([date, year]))
                            
                            1/0
                    elif len(vals) == 1:
                        date = vals[0]
                        try:
                            date = getDateTime(", ".join([date, year]))
                        except:
                            print(vals)
                            1/0

                    if not isDate(date):
                        raise ValueError("Could not form date for {0}".format(date))
                        
                    if data.get(artist) is None:
                        data[artist] = {}
                        print("Artist: {0}".format(artist))
                    if data[artist].get(name) is None:
                        data[artist][name] = {}
                        #print("\tSong: {0} ({1})".format(name, len(data[artist])))
                    if data[artist][name].get(category) is None:
                        data[artist][name][category] = []
                    data[artist][name][category].append([date])

                        
                        
            print("Found {0} artists".format(len(data)))
            
            
for artist,artistData in data.items():
    for name,nameData in artistData.items():
        for category,categoryData in nameData.items():
            data[artist][name][category] = [min(categoryData),max(categoryData)]
            

savedir  = join(basedir, "results")
savename = join(savedir, "billboard.p")
saveFile(ifile=savename, idata=data, debug=True)

Artist: Brook Benton
Artist: Connie Francis
Artist: Joe Dowell
Artist: The Highwaymen
Artist: Bob Moore and His Orch.
Artist: Sue Thompson
Artist: Jimmy Dean
Artist: Henry Mancini
Found 8 artists
Artist: The Lettermen
Artist: Elvis Presley
Artist: Burl Ives
Artist: Kenny Ball and his Jazzmen
Artist: Mr. Acker Bilk
Artist: Ray Charles
Artist: David Rose
Artist: Bobby Vinton
Artist: Nat King Cole
Artist: Frank Ifield
Artist: Gene Pitney
Artist: Brenda Lee
Artist: Steve Lawrence
Found 21 artists
Artist: The Rooftop Singers
Artist: The Cascades
Artist: Skeeter Davis
Artist: Andy Williams
Artist: Peter, Paul & Mary
Artist: Al Martino
Artist: Kyu Sakamoto
Artist: Rolf Harris
Artist: The Village Stompers
Artist: Dale & Grace
Artist: The Singing Nun (Soeur Sourire)
Found 32 artists
Artist: Rick Nelson
Artist: Al Hirt
Artist: Diane Renay
Artist: Louis Armstrong And The All Stars
Artist: The Ray Charles Singers
Artist: Barbra Streisand
Artist: Getz/Gilberto
Artist: Dean Martin
Artist: Gale Garne

Artist: Melissa Etheridge
Artist: Martin Page
Artist: The Rembrandts
Artist: Seal
Artist: Sophie B. Hawkins
Artist: Mariah Carey & Boyz II Men
Found 340 artists
Artist: Journey
Found 341 artists
Artist: Toni Braxton
Artist: Kenny Loggins
Artist: Jewel
Artist: Bob Carlisle
Artist: Shawn Colvin
Artist: LeAnn Rimes
Found 347 artists
Artist: Savage Garden
Artist: Shania Twain
Artist: Backstreet Boys
Artist: R. Kelly & Celine Dion
Found 351 artists
Artist: Sarah McLachlan
Found 352 artists
Artist: Faith Hill
Artist: Marc Anthony
Artist: Don Henley
Artist: BBMak
Artist: Huey Lewis & Gwyneth Paltrow
Artist: 'N Sync
Found 358 artists
Artist: NewSong
Artist: Lee Ann Womack
Artist: Dido
Artist: Enya
Artist: matchbox twenty
Artist: Enrique Iglesias
Found 364 artists
Artist: Jim Brickman Featuring Rebecca Lynn Howard
Artist: Josh Groban
Artist: Daryl Hall John Oates
Artist: Vanessa Carlton
Found 368 artists
Artist: Dixie Chicks
Artist: Santana Featuring Michelle Branch
Artist: Christina Aguilera
A

Artist: Siouxsie & The Banshees
Artist: Big Audio
Artist: Psychedelic Furs
Found 672 artists
Artist: Julian Cope
Artist: Lou Reed
Artist: The Replacements
Artist: Elvis Costello
Artist: XTC
Artist: The Cure
Artist: Love And Rockets
Artist: Public Image Ltd.
Artist: The B-52s
Artist: Hoodoo Gurus
Artist: Tears For Fears
Artist: Camper van Beethoven
Artist: Ian McCulloch
Artist: Kate Bush
Artist: The Jesus And Mary Chain
Found 687 artists
Artist: The Psychedelic Furs
Artist: Peter Murphy
Artist: Sinead O'Connor
Artist: Midnight Oil
Artist: The Church
Artist: The Sundays
Artist: World Party
Artist: Concrete Blonde
Artist: Gene Loves Jezebel
Artist: David J
Artist: Jane's Addiction
Artist: The Railway Children
Artist: INXS
Artist: Sisters Of Mercy
Found 701 artists
Artist: Happy Mondays
Artist: Sting
Artist: Jesus Jones
Artist: Simple Minds
Artist: Electronic
Artist: Robyn Hitchcock
Artist: Nirvana
Found 708 artists
Artist: Talking Heads
Artist: The Sugarcubes
Artist: Cracker
Artist: The C

Artist: Michael Jackson
Artist: The Police
Artist: Quiet Riot
Found 991 artists
Artist: Prince And The Revolution
Found 992 artists
Artist: Wham!
Artist: John Fogerty
Artist: Dire Straits
Found 995 artists
Artist: Mr. Mister
Artist: Van Halen
Artist: Patti LaBelle
Found 998 artists
Found 998 artists
Artist: Def Leppard
Artist: Guns N' Roses
Found 1000 artists
Artist: Bobby Brown
Artist: Debbie Gibson
Artist: Tone-Loc
Artist: Fine Young Cannibals
Artist: New Kids On The Block
Artist: Milli Vanilli
Artist: Motley Crue
Found 1007 artists
Artist: M.C. Hammer
Artist: Vanilla Ice
Found 1009 artists
Artist: N.W.A
Artist: Skid Row
Artist: Garth Brooks
Found 1012 artists
Artist: Kris Kross
Artist: The Black Crowes
Artist: Billy Ray Cyrus
Artist: Ice Cube
Found 1016 artists
Artist: Aerosmith
Artist: Cypress Hill
Artist: Meat Loaf
Artist: Snoop Doggy Dogg
Found 1020 artists
Artist: John Michael Montgomery
Artist: Soundgarden
Artist: Ace Of Base
Artist: Pantera
Artist: Tim McGraw
Found 1025 artist

Artist: Steve Earle And The Dukes
Artist: JJ Grey & MOFRO
Artist: Beth Hart
Artist: Samantha Fish
Artist: Bobby Messano
Artist: Billy Gibbons And The BFG's
Found 1308 artists
Artist: The Rides
Artist: Devon Allman
Artist: Mike Zito
Found 1311 artists
Artist: Curtis Knight Featuring Jimi Hendrix
Artist: Robert Cray & Hi Rhythm
Artist: Taj Mahal & Keb' Mo'
Artist: Chuck Berry
Artist: ZZ Ward
Artist: Savoy Brown
Artist: Walter Trout
Artist: Black Stone Cherry
Found 1319 artists
Artist: Tinsley Ellis
Artist: Danielle Nicole
Artist: Victor Wainwright And The Train
Artist: Paul Thorn
Artist: Ben Harper And Charlie Musselwhite
Artist: Malina Moye
Artist: Dave Alvin And Jimmie Dale Gilmore
Artist: Larkin Poe
Found 1327 artists
Found 1327 artists
Artist: Our Lady Peace
Artist: The Tragically Hip
Artist: Aqua
Found 1330 artists
Found 1330 artists
Artist: Matthew Good Band
Artist: Lou Bega
Found 1332 artists
Found 1332 artists
Artist: Diana Krall
Found 1333 artists
Artist: Great Big Sea
Artist: R

...

Artist: Ruby Turner
Artist: Stacy Lattisaw & Johnny Gill
Artist: Troop
Artist: After 7
Artist: Quincy Jones With Tevin Campbell
Artist: The Time
Artist: Samuelle
Found 5722 artists
Artist: The Rude Boys
Artist: Christopher Williams
Artist: Phil Perry
Artist: Lisa Fischer
Artist: Damian Dame
Artist: D.J. Jazzy Jeff & The Fresh Prince
Artist: Phyllis Hyman
Artist: Tracie Spencer
Artist: Shanice
Found 5731 artists
Artist: Gerald Levert (Duet With Eddie Levert)
Artist: Glenn Jones
Artist: R. Kelly & Public Announcement
Artist: Luther Vandross And Janet Jackson With BBD And Ralph Tresvant
Found 5735 artists
Artist: Tag Team
Artist: Ice Cube Featuring Das EFX
Artist: DRS
Found 5738 artists
Found 5738 artists
Artist: Method Man Featuring Mary J. Blige
Found 5739 artists
Artist: Az Yet
Found 5740 artists
Found 5740 artists
Artist: Montell Jordan Feat. Master P & Silkk The Shocker
Artist: Janet Featuring BLACKstreet
Artist: Dru Hill Featuring Redman
Found 5743 artists
Artist: Busta Rhymes Featu

Found 5933 artists
Artist: Passafire
Artist: J Boog
Artist: The Green
Artist: Jimmy Cliff
Found 5937 artists
Artist: SOJA
Found 5938 artists
Artist: Fiji
Artist: Fortunate Youth
Artist: Christafari
Artist: Snoop Lion
Artist: Musiq Soulchild/Syleena Johnson
Artist: Gyptian
Artist: Makua Rothman
Found 5945 artists
Artist: Papa San
Artist: Tarrus Riley
Artist: Chronixx
Artist: Tribal Seeds
Artist: Giant Panda Guerilla Dub Squad
Artist: Etana
Found 5951 artists
Artist: New Kingston
Artist: Gavinchi Brown
Artist: Protoje
Artist: The Expanders
Artist: Jah Cure
Found 5956 artists
Artist: Josh Heinrichs
Artist: Alkaline
Artist: Roots Of Creation
Artist: HIRIE
Artist: The Frightnrs
Found 5961 artists
Artist: Landon McNamara
Artist: Common Kings
Artist: Queen Ifrica
Artist: Tenelle
Artist: Iya Terra
Artist: Satsang
Artist: Young Shanty
Artist: The Expendables
Artist: Mystic Roots Band
Artist: Xyclone
Artist: Perfect Giddimani
Artist: IamStylezMusic
Artist: Ooklah The Moc
Artist: Jesse Royal
Arti

Found 6122 artists
Found 6122 artists
Found 6122 artists
Artist: Kurt Cobain
Found 6123 artists
Found 6123 artists
Found 6123 artists
Found 6123 artists
Artist: Ylvis
Found 6124 artists
Found 6124 artists
Found 6124 artists
Found 6124 artists
Found 6124 artists
Found 6124 artists
Found 6124 artists
Artist: Built To Spill
Artist: J.J. Cale & Eric Clapton
Found 6126 artists
Artist: Ben Harper And The Innocent Criminals
Found 6127 artists
Artist: Ryan Adams & The Cardinals
Found 6128 artists
Artist: Animal Collective
Artist: Yeah Yeah Yeahs
Artist: Sonic Youth
Found 6131 artists
Artist: John Legend & The Roots
Artist: Fistful Of Mercy
Artist: Belle And Sebastian
Found 6134 artists
Artist: Chris Cornell
Found 6135 artists
Artist: Sigur Ros
Artist: Neil Young & Crazy Horse
Found 6137 artists
Artist: Yo La Tengo
Artist: Jim James
Artist: Nick Cave & The Bad Seeds
Artist: Kurt Vile
Artist: Washed Out
Artist: Elvis Costello And The Roots
Found 6143 artists
Artist: Stephen Malkmus And The Jicks

Artist: Valentino Khan
Artist: Primal Scream
Artist: Seinabo Sey
Artist: Jon Bellion
Artist: Troy Ave
Artist: Bebe Rexha Featuring Nicki Minaj
Artist: Kiiara
Artist: Marshmello Featuring Omar LinX
Artist: Transviolet
Artist: J Dilla Featuring Nas
Artist: Cash Cash Featuring Sofia Reyes
Artist: Sampha
Artist: Young Greatness
Artist: Alan Walker
Artist: Snakehips Featuring Zayn
Artist: Francis & The Lights Featuring Bon Iver
Artist: Mura Masa Featuring A$AP Rocky
Found 6391 artists
Artist: Julia Michaels
Artist: Syd
Artist: Gallant x Tablo x Eric Nam
Artist: Bibi Bourelly
Artist: Stargate Featuring P!nk & Sia
Artist: Cheat Codes Featuring Demi Lovato
Artist: SEVENTEEN
Artist: Lost Kings Featuring Ally Brooke & A$AP Ferg
Artist: Jax Jones Featuring Demi Lovato & Stefflon Don
Artist: GoldLink Featuring Brent Faiyaz & Shy Glizzy
Artist: Daniel Caesar Featuring Kali Uchis
Artist: RiceGum
Found 6403 artists
Artist: Demi Lovato Featuring Cher Lloyd
Artist: Naughty Boy Featuring Sam Romans
Arti

NameError: name 'saveFile' is not defined

# Load Data

In [5]:
savedir  = join(basedir, "results")
savename = join(savedir, "billboard.p")
chartdata = getFile(savename, debug=True)

Loading data from /Users/tgadfort/Documents/code/charts/results/billboard.p
  --> This file is 1.1MB.
Loading /Users/tgadfort/Documents/code/charts/results/billboard.p


# Show Data

In [21]:
for artist in sorted(chartdata.keys()):
    artistData = data[artist]
    years = []
    for name,nameData in artistData.items():
        for chart,chartData in nameData.items():
            years += chartData[0]
            years += chartData[1]
            
    
    
    print("{0: <75}{1} <-> {2}".format(artist,min(years).year,max(years).year))
    continue
    for name,nameData in artistData.items():
        print("\t",name,'\t',nameData)
    break

"Little Esther" Phillips                                                   1962 <-> 1963
"Little" Jimmy Dickens                                                     1965 <-> 1965
"Weird Al" Yankovic                                                        2006 <-> 2017
$uicideBoy$                                                                2018 <-> 2018
'N Sync                                                                    1999 <-> 2001
(Game Music)                                                               2016 <-> 2016
(The Preacher) Bobby Womack                                                1972 <-> 1972
10 Years                                                                   2006 <-> 2008
10,000 Maniacs                                                             1992 <-> 1992
112                                                                        1997 <-> 2001
2 Chainz                                                                   2012 <-> 2017
2 Chainz Featuring Dr

Des'ree                                                                    1994 <-> 1995
Descendents                                                                2016 <-> 2016
Desert                                                                     2001 <-> 2001
Desiigner                                                                  2016 <-> 2016
Desiya                                                                     1992 <-> 1992
Deskee                                                                     1990 <-> 1990
Destiny's Child                                                            1998 <-> 2005
Destiny's Child Featuring T.I. & Lil Wayne                                 2005 <-> 2005
Destroy Rebuild Until God Shows                                            2011 <-> 2011
Dethklok                                                                   2007 <-> 2012
Dev                                                                        2011 <-> 2011
Deva Premal          

Janet Jackson                                                              1986 <-> 2006
Janie Frickie                                                              1982 <-> 1986
Janis Ian                                                                  1975 <-> 1975
Janis Joplin                                                               1971 <-> 1971
Jann Arden                                                                 1996 <-> 1996
Japandroids                                                                2012 <-> 2012
Jaron And The Long Road To Love                                            2010 <-> 2010
Jarreau                                                                    1981 <-> 1981
Jarren Benton                                                              2015 <-> 2015
Jarrod Alonge                                                              2015 <-> 2015
Jars Of Clay                                                               1996 <-> 2009
Jason Aldean         

Mos Def                                                                    2004 <-> 2004
Mos Def & Pharoahe Monch Featuring Nate Dogg                               2001 <-> 2001
Motion City Soundtrack                                                     2007 <-> 2007
Motionless In White                                                        2014 <-> 2017
Motley Crue                                                                1989 <-> 2008
Mountain Faith                                                             2015 <-> 2015
Mouth Music                                                                1991 <-> 1991
Mozart La Para Featuring Sharlene & Nacho                                  2016 <-> 2016
Mr. Acker Bilk                                                             1962 <-> 1962
Mr. Big                                                                    1992 <-> 1992
Mr. Cheeks                                                                 2002 <-> 2002
Mr. Children         

Tedeschi Trucks Band                                                       2011 <-> 2017
Teena Marie                                                                1988 <-> 1988
Teenage Fanclub                                                            1992 <-> 2016
Tegan And Sara                                                             2013 <-> 2013
Temper                                                                     1984 <-> 1984
Ten City                                                                   1989 <-> 1989
Tenacious D                                                                2006 <-> 2018
Tenelle                                                                    2017 <-> 2017
Tennis                                                                     2012 <-> 2012
Tenth Avenue North                                                         2010 <-> 2018
Tercer Cielo                                                               2009 <-> 2009
Terence Trent D'Arby 

In [11]:
from searchUtils import findNearest
artists = sorted(chartdata.keys())
discArtists = discdata.keys()
for artist in artists: 
    discID = discdata.get(artist)
    if discID is None:
        discID = findNearest(artist, discArtists, 1, 0.95)
    print("{0: <75}{1}".format(artist,discID))

"Little Esther" Phillips                                                   []
"Little" Jimmy Dickens                                                     ['Little Jimmy Dickens']
"Weird Al" Yankovic                                                        ['259422']
$uicideBoy$                                                                []
'N Sync                                                                    []
(Game Music)                                                               []
(The Preacher) Bobby Womack                                                []
10 Years                                                                   ['991804']
10,000 Maniacs                                                             ['63486']
112                                                                        ['113470']
2 Chainz                                                                   ['2523290']
2 Chainz Featuring Drake                                                   []
2 

Adam Hawley Featuring Eric Darius                                          []
Adam Hawley Featuring Eugue Groove                                         []
Adam Hawley Featuring Julian Vaughan                                       []
Adam Hawley Featuring Marcus Anderson                                      []
Adam Lambert                                                               ['1517742', '646266']
Adan Chalino Sanchez                                                       []
Adele                                                                      ['819015']
Adelitas Way                                                               []
Adeva                                                                      ['15169']
Adina Howard                                                               ['54186']
Adolescent's Orquesta                                                      ['1151942']
Adore Delano                                                               ['3829833']
Aeros

Alta Consigna                                                              []
Altar Featuring Jeanie Tracy                                               []
Alter Bridge                                                               ['713239']
Alter Bridge Featuring The Parallax Orchestra                              []
Alvaro Soler Featuring Jennifer Lopez                                      []
Alvaro Torres                                                              []
Alvin And The Chipmunks                                                    []
Amadou & Mariam                                                            ['144288']
Amanda Cook                                                                []
Amanda Miguel                                                              ['1006079']
Amanda Perez                                                               ['404026']
Amanda Shires                                                              ['1104134']
Amaranthe             

Antonio Barullo Featuring J Alvarez & Flex                                 []
Antonio Orozco                                                             ['1039010']
Anuel AA                                                                   []
Aoi Teshima                                                                []
Aphex Twin                                                                 ['45']
Apocalyptica Featuring Adam Gontier                                        []
Aqua                                                                       ['199132', '38862', '666818', '1177966', '214678']
Aqualung                                                                   ['87185']
Arashi                                                                     ['41911', '2805345']
Arc Angels                                                                 ['548286']
Arcade Fire                                                                ['281232']
Arcangel                                

B.B. King & Eric Clapton                                                   ['B.B King & Eric Clapton']
B.G.                                                                       ['137604', '5493893']
B.G. And The Chopper City Boyz                                             []
B.G. The Prince Of Rap                                                     ['42554']
B.J. Thomas                                                                ['182608']
B.T. Express                                                               ['170575', '485149']
B.W. Stevenson                                                             ['B. W. Stevenson']
B.o.B                                                                      ['901031']
B.o.B Featuring Bruno Mars                                                 []
B.o.B Featuring Hayley Williams                                            []
B.o.B Featuring Lil Wayne                                                  []
B.o.B Featuring Taylor Swift            

...

Twelve Girls Band                                                          []
Twila Paris                                                                ['431778']
Twista                                                                     ['133869']
Twista Featuring Kanye West & Jamie Foxx                                   []
Twiztid                                                                    ['85888']
Two Feet                                                                   []
Two Steps From Hell                                                        ['1903958']
Ty Herndon                                                                 ['1323588']
Ty Segall                                                                  ['1265284']
Tycho                                                                      ['71827']
Tye Tribbett                                                               ['346746']
Tye Tribbett & G.A.                                                        []

Vertical Church Band                                                       []
Vertical Horizon                                                           ['252680']
Veruca Salt                                                                ['28974']
Vic Latino & David Waxman                                                  []
Vic Mensa                                                                  ['3268577']
Vicente Fernandez                                                          ['419130']
Vicente Y Alejandro Fernandez                                              []
Vicious                                                                    ['245528', '379137']
Vicki Lawrence                                                             ['988977']
Vickie Winans                                                              ['350147']
Victor Calderone                                                           ['13734']
Victor Calderone Featuring Deborah Cooper                            

Will Pan                                                                   []
Will Smith                                                                 ['74169', '5248678', '5111178', '5059266', '4238912', '3407326', '3228086', '3007273', '2852309', '1510281', '1449286', '1100591', '843114', '812833', '394201']
Will Smith Featuring Dru Hill & Kool Mo Dee                                []
Will To Power                                                              ['43744']
Willam                                                                     ['3197045']
William Becton And Friends                                                 []
William Bell                                                               ['88689', '4684867', '4479526', '4248684', '4181147', '2561365', '1009694', '630867']
William Clark Green                                                        ['3899327']
William Control                                                            ['1265516']
William DeVaughn       

Yo-Yo Featuring Ice Cube                                                   []
Yo-Yo Ma                                                                   ['276145']
Yo-Yo Ma & The Silk Road Ensemble                                          []
Yo-Yo Ma / Chris Thile / Edgar Meyer                                       []
Yo-Yo Ma / Kathryn Stott                                                   []
Yo-Yo Ma/Edgar Meyer/Mark O'Connor                                         []
Yo-Yo Ma/Stuart Duncan/Edgar Meyer/Chris Thile                             []
Yoan                                                                       ['4632884', '3840495', '2129190', '1798262']
Yolanda Adams                                                              ['118531']
Yolanda Be Cool & Dcup                                                     []
Yolandita Monge                                                            ['1204920']
Yonder Mountain String Band                                                

f(x)                                                                       []
for KING & COUNTRY                                                         []
forever JONES                                                              []
fun.                                                                       []
fun. Featuring Janelle Monae                                               []
iKON                                                                       []
iio                                                                        []
iwrestledabearonce                                                         ['1428409']
k.d. lang                                                                  ['110570']
k.flay                                                                     []
lovelytheband                                                              []
matchbox 20                                                                []
matchbox twenty                                